In [1]:
import pandas as pd
import numpy as np
import sklearn

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from surprise import KNNWithMeans
from surprise import accuracy
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

In [2]:
links = pd.read_csv('C:/Users/Subhadri/Desktop/Data Science - Term 1/Machine Learning/Assignment 3/ml-latest-small/links.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [3]:
links.shape

(9742, 3)

In [4]:
links['movieId'].nunique()

9742

In [5]:
movies = pd.read_csv('C:/Users/Subhadri/Desktop/Data Science - Term 1/Machine Learning/Assignment 3/ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies.shape

(9742, 3)

In [7]:
movies['movieId'].nunique()

9742

In [8]:
ratings = pd.read_csv('C:/Users/Subhadri/Desktop/Data Science - Term 1/Machine Learning/Assignment 3/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
ratings.shape

(100836, 4)

In [10]:
ratings['userId'].nunique()

610

In [11]:
ratings['userId'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [12]:
ratings['movieId'].nunique()

9724

In [13]:
ratings['rating'].unique()

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

In [14]:
tags = pd.read_csv('C:/Users/Subhadri/Desktop/Data Science - Term 1/Machine Learning/Assignment 3/ml-latest-small/tags.csv')
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [15]:
tags.shape

(3683, 4)

In [16]:
tags['movieId'].nunique()

1572

In [17]:
movie_ratings = pd.merge(movies, ratings, on = 'movieId')
movie_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [18]:
movie_ratings_pivot = (movie_ratings.pivot_table(index = 'title', columns = 'userId', values = 'rating').fillna(0))
movie_ratings_pivot.shape

(9719, 610)

In [19]:
movie_ratings_pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
movie_ratings_matrix = csr_matrix(movie_ratings_pivot.values)
movie_ratings_matrix

<9719x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100832 stored elements in Compressed Sparse Row format>

In [21]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_ratings_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

### 1. Create recommender system using ratings.csv file.

In [22]:
query = input('Enter a movie name:')
distances, indices = model_knn.kneighbors(movie_ratings_pivot.loc[query, :].values.reshape(1,-1), n_neighbors = 11)
print(distances)
print(indices)

Enter a movie name:'Salem's Lot (2004)
[[0.         0.         0.14250707 0.2407434  0.35549661 0.42501109
  0.44662843 0.47510934 0.47585758 0.49108664 0.5       ]]
[[  3 350   4 197  14 144  99 348 219 456 474]]


In [23]:
print('Recommendations for', query, '\n'.format(query))
for i in indices.flatten()[1:]:
    print(movie_ratings_pivot.index[i])

Recommendations for 'Salem's Lot (2004) 

All This, and Heaven Too (1940)
'Til There Was You (1997)
Absence of Malice (1981)
10 (1979)
84 Charing Cross Road (1987)
3 Women (Three Women) (1977)
All That Heaven Allows (1955)
Adam's Rib (1949)
Amityville Horror, The (2005)
Anatomy of a Murder (1959)


### 2. Display predicted rating for a particular userID and MovieID combinations (both taken as keyboard input).

In [24]:
reader = Reader(rating_scale=(0.5,5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

[trainset, testset] = train_test_split(data, test_size= .15, shuffle= True)


recom =KNNWithMeans(k=9, sim_options={'name': 'cosine', 'user_based': True})
recom.fit(data.build_full_trainset()) # using full training set
test_pred = recom.test(testset)
RMSE = accuracy.rmse(test_pred)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.6993


In [25]:
movie_id = int(input('Enter a movie_id: '))
user_id = int(input('Enter a user_id: '))
prediction = recom.predict(movie_id, user_id)
print("The rating value for movie {} by user {} is %.2f ".format(movie_id, user_id) % prediction[3])

Enter a movie_id: 16
Enter a user_id: 25
The rating value for movie 16 by user 25 is 4.28 


### 3. Recommend a user to watch top 10 movies, which has not watched yet (as per his rating predictions). Take userID as a keyboard input. Fix the threshold rating as 2.5.

In [26]:
data_pivot = (ratings.pivot_table(index = 'movieId', columns = 'userId', values = 'rating').fillna(0))
data_pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
counter = 0
user_id = int(input('Enter a user_id: '))
list1 = []
list2 = []
for i in data_pivot[user_id]:
    if i==0:
        prediction = recom.predict(user_id,data_pivot.loc[:,user_id].keys()[counter])
        if prediction[3]>=2.5:
            list1.append(prediction[1])
            list2.append(prediction[3])
    counter += 1
data_frame = pd.DataFrame()
data_frame['movieId'] = list1
data_frame['Ratings'] = list2
data = data_frame.merge(movies.iloc[:,:2], on = 'movieId')
top_10_movies = data.sort_values(by = 'Ratings', ascending = False).head(10)
top_10_movies

Enter a user_id: 31


,movieId,Ratings,title
1409,2037,5.0,Candleshoe (1977)
8168,121372,5.0,Bill Burr: Let It Go (2010)
8016,114265,5.0,Laggies (2014)
8050,115727,5.0,Crippled Avengers (Can que) (Return of the 5 D...
8052,115828,5.0,Copenhagen (2014)
4179,6611,5.0,Umberto D. (1952)
8090,117531,5.0,Watermark (2014)
4138,6545,5.0,I Capture the Castle (2003)
8108,118270,5.0,Hellbenders (2012)
4129,6533,5.0,"What's Up, Doc? (1972)"


### 4. Display the MovieID, IMDB ID, Average User Rating (excluding predictions), genres and tag of all the movies found in Step 3 as a data frame.

In [28]:
average_user_rating=movie_ratings.groupby('movieId')['rating'].mean().reset_index()
average_user_rating.columns = ['movieId', 'average_rating']
average_user_rating.head()

,movieId,average_rating
0,1,3.920930
1,2,3.431818
2,3,3.259615
3,4,2.357143
4,5,3.071429


In [29]:
x = top_10_movies.merge(movies)
y = x.merge(links.iloc[:,:2], on = 'movieId')
z = y.merge(average_user_rating)
final_data_frame = z.merge(tags.iloc[:,1:3], on = 'movieId', how = 'left').drop_duplicates()

In [30]:
final_data_frame

,movieId,Ratings,title,genres,imdbId,average_rating,tag
0,2037,5.0,Candleshoe (1977),Adventure|Children|Comedy,75807,4.0,NaN
1,121372,5.0,Bill Burr: Let It Go (2010),Comedy,1717578,4.5,NaN
2,114265,5.0,Laggies (2014),Comedy|Romance,2034031,5.0,NaN
3,115727,5.0,Crippled Avengers (Can que) (Return of the 5 D...,Action|Adventure,77292,5.0,NaN
4,115828,5.0,Copenhagen (2014),Adventure|Drama|Romance,2459156,4.5,NaN
5,6611,5.0,Umberto D. (1952),Drama,45274,5.0,NaN
6,117531,5.0,Watermark (2014),Documentary,3106868,5.0,NaN
7,6545,5.0,I Capture the Castle (2003),Drama|Romance,300015,4.5,Dodie Smith
8,118270,5.0,Hellbenders (2012),Comedy|Horror|Thriller,1865393,5.0,NaN
9,6533,5.0,"What's Up, Doc? (1972)",Comedy,69495,4.5,NaN
